# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4892, train_loss: 2.35166407, train_accuracy: 0.2266, test_Accuracy: 0.1755
Epoch: [ 0] [    1/  468] time: 0.7493, train_loss: 2.31816411, train_accuracy: 0.3281, test_Accuracy: 0.2692
Epoch: [ 0] [    2/  468] time: 1.0163, train_loss: 2.27257347, train_accuracy: 0.3047, test_Accuracy: 0.3348
Epoch: [ 0] [    3/  468] time: 1.2659, train_loss: 2.30772448, train_accuracy: 0.3906, test_Accuracy: 0.3769
Epoch: [ 0] [    4/  468] time: 1.5214, train_loss: 2.30245805, train_accuracy: 0.3906, test_Accuracy: 0.4140
Epoch: [ 0] [    5/  468] time: 1.7696, train_loss: 2.21979237, train_accuracy: 0.4141, test_Accuracy: 0.4624
Epoch: [ 0] [    6/  468] time: 2.0366, train_loss: 2.22849751, train_accuracy: 0.5078, test_Accuracy: 0.5241
Epoch: [ 0] [    7/  468] time: 2.2963, train_loss: 2.17310953, train_accuracy: 0.6016, test_Accuracy: 0.5381
Epoch: [ 0] [    8/  468] time: 2.5491,

Epoch: [ 0] [   74/  468] time: 18.9003, train_loss: 0.71976399, train_accuracy: 0.8516, test_Accuracy: 0.8722
Epoch: [ 0] [   75/  468] time: 19.1427, train_loss: 0.62141907, train_accuracy: 0.8984, test_Accuracy: 0.8755
Epoch: [ 0] [   76/  468] time: 19.3877, train_loss: 0.46489429, train_accuracy: 0.9062, test_Accuracy: 0.8752
Epoch: [ 0] [   77/  468] time: 19.6312, train_loss: 0.56345868, train_accuracy: 0.8672, test_Accuracy: 0.8741
Epoch: [ 0] [   78/  468] time: 19.8710, train_loss: 0.57509935, train_accuracy: 0.8906, test_Accuracy: 0.8757
Epoch: [ 0] [   79/  468] time: 20.1191, train_loss: 0.69914323, train_accuracy: 0.8906, test_Accuracy: 0.8769
Epoch: [ 0] [   80/  468] time: 20.3568, train_loss: 0.68760693, train_accuracy: 0.8984, test_Accuracy: 0.8827
Epoch: [ 0] [   81/  468] time: 20.6043, train_loss: 0.47655237, train_accuracy: 0.9062, test_Accuracy: 0.8904
Epoch: [ 0] [   82/  468] time: 20.8731, train_loss: 0.42906165, train_accuracy: 0.8984, test_Accuracy: 0.8948
E

Epoch: [ 0] [  148/  468] time: 37.0641, train_loss: 0.40230501, train_accuracy: 0.8672, test_Accuracy: 0.9222
Epoch: [ 0] [  149/  468] time: 37.3070, train_loss: 0.34811944, train_accuracy: 0.9375, test_Accuracy: 0.9216
Epoch: [ 0] [  150/  468] time: 37.5529, train_loss: 0.35084498, train_accuracy: 0.8984, test_Accuracy: 0.9216
Epoch: [ 0] [  151/  468] time: 37.8077, train_loss: 0.24494763, train_accuracy: 0.9531, test_Accuracy: 0.9219
Epoch: [ 0] [  152/  468] time: 38.0480, train_loss: 0.37574947, train_accuracy: 0.9453, test_Accuracy: 0.9232
Epoch: [ 0] [  153/  468] time: 38.2901, train_loss: 0.28041494, train_accuracy: 0.9531, test_Accuracy: 0.9251
Epoch: [ 0] [  154/  468] time: 38.5334, train_loss: 0.37605911, train_accuracy: 0.9688, test_Accuracy: 0.9266
Epoch: [ 0] [  155/  468] time: 38.7816, train_loss: 0.46547490, train_accuracy: 0.8906, test_Accuracy: 0.9272
Epoch: [ 0] [  156/  468] time: 39.0264, train_loss: 0.35544017, train_accuracy: 0.9688, test_Accuracy: 0.9281
E

Epoch: [ 0] [  222/  468] time: 55.2946, train_loss: 0.23482168, train_accuracy: 0.9766, test_Accuracy: 0.9379
Epoch: [ 0] [  223/  468] time: 55.5344, train_loss: 0.30466810, train_accuracy: 0.9297, test_Accuracy: 0.9367
Epoch: [ 0] [  224/  468] time: 55.7768, train_loss: 0.35650253, train_accuracy: 0.9297, test_Accuracy: 0.9360
Epoch: [ 0] [  225/  468] time: 56.0207, train_loss: 0.34423637, train_accuracy: 0.9453, test_Accuracy: 0.9365
Epoch: [ 0] [  226/  468] time: 56.2686, train_loss: 0.30451143, train_accuracy: 0.9375, test_Accuracy: 0.9372
Epoch: [ 0] [  227/  468] time: 56.5169, train_loss: 0.43092716, train_accuracy: 0.9219, test_Accuracy: 0.9383
Epoch: [ 0] [  228/  468] time: 56.7660, train_loss: 0.31617412, train_accuracy: 0.9531, test_Accuracy: 0.9394
Epoch: [ 0] [  229/  468] time: 57.0209, train_loss: 0.35764739, train_accuracy: 0.9297, test_Accuracy: 0.9397
Epoch: [ 0] [  230/  468] time: 57.2646, train_loss: 0.39516395, train_accuracy: 0.9453, test_Accuracy: 0.9412
E

Epoch: [ 0] [  296/  468] time: 73.3030, train_loss: 0.31580269, train_accuracy: 0.9531, test_Accuracy: 0.9459
Epoch: [ 0] [  297/  468] time: 73.5446, train_loss: 0.37518737, train_accuracy: 0.9375, test_Accuracy: 0.9460
Epoch: [ 0] [  298/  468] time: 73.7842, train_loss: 0.27873743, train_accuracy: 0.9297, test_Accuracy: 0.9447
Epoch: [ 0] [  299/  468] time: 74.0228, train_loss: 0.39344108, train_accuracy: 0.9453, test_Accuracy: 0.9447
Epoch: [ 0] [  300/  468] time: 74.2611, train_loss: 0.32158673, train_accuracy: 0.9531, test_Accuracy: 0.9443
Epoch: [ 0] [  301/  468] time: 74.5017, train_loss: 0.27024978, train_accuracy: 0.9609, test_Accuracy: 0.9445
Epoch: [ 0] [  302/  468] time: 74.7459, train_loss: 0.18463610, train_accuracy: 0.9766, test_Accuracy: 0.9454
Epoch: [ 0] [  303/  468] time: 74.9879, train_loss: 0.30676609, train_accuracy: 0.9219, test_Accuracy: 0.9456
Epoch: [ 0] [  304/  468] time: 75.2311, train_loss: 0.23281193, train_accuracy: 0.9609, test_Accuracy: 0.9441
E

Epoch: [ 0] [  370/  468] time: 91.3308, train_loss: 0.19224104, train_accuracy: 0.9609, test_Accuracy: 0.9536
Epoch: [ 0] [  371/  468] time: 91.5643, train_loss: 0.16831174, train_accuracy: 0.9531, test_Accuracy: 0.9539
Epoch: [ 0] [  372/  468] time: 91.8032, train_loss: 0.18280905, train_accuracy: 0.9766, test_Accuracy: 0.9542
Epoch: [ 0] [  373/  468] time: 92.0412, train_loss: 0.15925537, train_accuracy: 0.9766, test_Accuracy: 0.9546
Epoch: [ 0] [  374/  468] time: 92.2828, train_loss: 0.25645590, train_accuracy: 0.9609, test_Accuracy: 0.9546
Epoch: [ 0] [  375/  468] time: 92.5230, train_loss: 0.25996539, train_accuracy: 0.9453, test_Accuracy: 0.9548
Epoch: [ 0] [  376/  468] time: 92.7568, train_loss: 0.31770265, train_accuracy: 0.9375, test_Accuracy: 0.9551
Epoch: [ 0] [  377/  468] time: 92.9925, train_loss: 0.35784179, train_accuracy: 0.9531, test_Accuracy: 0.9548
Epoch: [ 0] [  378/  468] time: 93.2293, train_loss: 0.22785175, train_accuracy: 0.9453, test_Accuracy: 0.9554
E

Epoch: [ 0] [  444/  468] time: 109.1102, train_loss: 0.20996577, train_accuracy: 0.9688, test_Accuracy: 0.9574
Epoch: [ 0] [  445/  468] time: 109.3558, train_loss: 0.25112832, train_accuracy: 0.9453, test_Accuracy: 0.9577
Epoch: [ 0] [  446/  468] time: 109.5950, train_loss: 0.21572490, train_accuracy: 0.9297, test_Accuracy: 0.9584
Epoch: [ 0] [  447/  468] time: 109.8398, train_loss: 0.19312790, train_accuracy: 0.9766, test_Accuracy: 0.9577
Epoch: [ 0] [  448/  468] time: 110.0855, train_loss: 0.21895725, train_accuracy: 0.9766, test_Accuracy: 0.9572
Epoch: [ 0] [  449/  468] time: 110.3324, train_loss: 0.30323431, train_accuracy: 0.9609, test_Accuracy: 0.9573
Epoch: [ 0] [  450/  468] time: 110.5747, train_loss: 0.45926964, train_accuracy: 0.9375, test_Accuracy: 0.9579
Epoch: [ 0] [  451/  468] time: 110.8187, train_loss: 0.25460136, train_accuracy: 0.9766, test_Accuracy: 0.9584
Epoch: [ 0] [  452/  468] time: 111.0610, train_loss: 0.25121906, train_accuracy: 0.9609, test_Accuracy:

# Test accuracy : 95.91 %